In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import seaborn as sns
import math

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (11, 7)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna 

# Feel free to add all the libraries you need
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from geopy.distance import geodesic

In [2]:
df1 = pd.read_csv('data/ordinalencoded.csv', index_col=0)

In [3]:
df1 = df1[df1['target']!=0]
df = df1.drop(['dep_lat','dep_lon','arr_lat','arr_lon', 'day', 'year', 'arr_elevation', 'dep_elevation'], axis=1)
#df['status']=df['status']*df['status']*df['status']
#df['arr_iata']=df['arr_iata']*df['arr_iata']*df['arr_iata']



In [4]:
# define features and target
features = df.columns.tolist()
features.remove('target')
X = df[features]
y = df['target']

# LETS DO FEATURE ENGINEERING.....gonna square some shit 
# generate interaction features to the 3rd degree
p = PolynomialFeatures(interaction_only=False, include_bias=False, degree=1)
X_poly = p.fit_transform(X)
p.n_output_features_

20

In [5]:
X_train_unscaled, X_test, y_train, y_test = train_test_split(X_poly, y, random_state=12, test_size=0.8)

In [6]:
# standard scaler 
scaler = StandardScaler()
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_unscaled)

In [7]:
## feature engineering -- create function to pull out coefficients/features
## add a gradient descent
def lasso_optimization(x123_train):
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[], 'alpha':[], 'coeff_used':[], 'coefficients':[], 'intercept':[]}

    # initialize and train model with (default value) alpha = 0.5
    # for i in [0.1, 1, 10,100,1000, 10000, 33000,66000, 100000, 330000,660000,1000000, 3300000,6600000,10000000,33000000,66000000,100000000]:
    for i in [0.12,0.15]:

        model = Lasso(alpha=i, max_iter=int(2000))
        model.fit(x123_train,y_train)
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        ## use cross validation for 
        score = cross_validate(model, x123_train, y_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_mean_squared_error'), cv=3)
        
        ## pull out the scores
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_mean_squared_error'])
        coeff_used = np.sum(model.coef_!=0)
        coefficients = model.coef_
        intercept = model.intercept_

        dicKeys = ['mean_MAPE','mean_r2','mean_MSE','std_MAPE','std_r2','std_MSE','score','alpha','coeff_used', 'coefficients', 'intercept'] 
        dicValues = [mean_MAPE, mean_r2, mean_MSE, std_MAPE, std_r2, std_MSE, len(score), i, coeff_used, coefficients, intercept]
        for (keyi, vali) in zip(dicKeys, dicValues):
            eval_metrics_dictionary[keyi].append(vali)
        #eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        #eval_metrics_dictionary['mean_r2'].append(mean_r2)
        #eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        #eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        #eval_metrics_dictionary['std_r2'].append(std_r2)
        #eval_metrics_dictionary['std_MSE'].append(std_MSE)
        #eval_metrics_dictionary['score'].append(len(score))
        #eval_metrics_dictionary['alpha'].append(i)
        #eval_metrics_dictionary['coeff_used'].append(coeff_used)

    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    df_framework['combined_score'] = abs(abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']*df_framework['mean_MAPE'])
    df_bestvalue = df_framework[df_framework['combined_score']==min(df_framework['combined_score'])]
    return df_bestvalue
#return df_framework.sort_values('combined_score')

In [8]:
hi = lasso_optimization(X_train)
hi

,mean_r2,mean_MAPE,mean_MSE,std_r2,std_MAPE,std_MSE,score,alpha,coeff_used,coefficients,intercept,combined_score
0,0.018695,3.001788,21211.999672,0.003973,0.076588,1729.398338,5,0.12,14,"[31.71810494464688, 19.158839435453984, -11.74...",5.966296,62483.559933


In [9]:
# take list of coefficients - if more than zero 
# keep the name of the poly feature
# keep the non-zero poly features values
featurenames = list(p.get_feature_names_out(X.columns))
## pull the list of coefficients -- some funnny stuff needs to be done because of array stuff, basically reset_index
coefficient_list = list(hi['coefficients'].reset_index()['coefficients'][0]) 

### creature a dictionary of features and coefficients and then turn into dataframe
feature_coefficient_dict = {'feature':[], 'coefficient':[]}
for (feaI, coeI) in zip(featurenames, coefficient_list): 
    if coeI != 0: 
        feature_coefficient_dict['feature'].append(feaI)
        feature_coefficient_dict['coefficient'].append(coeI)
        
pd.DataFrame(feature_coefficient_dict)
feature_selection = feature_coefficient_dict['feature']
#### with the coefficients create a new np.array to 
ridge_Xtrain = np.array(pd.DataFrame(X_train, columns= p.get_feature_names_out(X.columns))[feature_selection])

In [10]:
#hi = lasso_optimization(Xtrain1)
pd.DataFrame(feature_coefficient_dict).sort_values('coefficient', ascending=False)

,feature,coefficient
13,std_time,50.006660
0,datop,31.718105
7,arr_country,29.675554
6,dep_country,22.960676
10,flight_time_in_min,22.733632
1,fltid,19.158839
4,month,12.544491
8,season,3.501632
5,dep_iata,-0.412945
3,ac,-5.196107


In [11]:
def ridge_optimization(number_of_trials):
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[]}
    ridge_opt_dictionary = { 'alpha': [], 'intercept' : [], 'tol': [], 'solver':[], 'no_of_coefficients':[]}

    def objective(trial):
        ## HYPER PARAMETER TUNING
        alpha = trial.suggest_float("alpha", 0, 100)
        intercept = trial.suggest_categorical("fit_intercept", [True, False])
        tol = trial.suggest_float("tol", 0.001, 1, log=True)
        solver = trial.suggest_categorical("solver", ["auto", "svd","cholesky", "lsqr", "saga", "sag"])

        ## MODEL SELECTION
        ## Create Model
        model = Ridge(alpha=alpha,fit_intercept=intercept,tol=tol,solver=solver)
        ## Fit Model
        model.fit(ridge_Xtrain, y_train)
        
        # Function parameters for cross_val_score; 
        #   - estimator - The model object to use to fit the data
        #   - X - The data to fit the model on
        #   - y - The target of the model
        #   - scoring - The error metric to use
        #   - cv - The number of splits to use
        
        # error score to use; 
        #   - accuracy
        #   - balanced_accuracy
        #   - roc_auc
        #   - f1
        #   - neg_mean_absolute_error
        #   - neg_root_mean_squared_error
        #   - r2
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        score = cross_validate(model, ridge_Xtrain, y_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_mean_squared_error'), cv=3)
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_mean_squared_error'])
        
        """
        y_pred_train= ridge_regressor.predict(X_test)
            
        Evaluation metrics;
        - R squared
        - MSE
        - RMSE
        - MAPE
        
        infun_MSE = mean_squared_error(y_test,y_pred_train)
        infun_RMSE = math.sqrt(infun_MSE)
        infun_r2 = r2_score(y_test, y_pred_train) 
        infun_MAPE = mean_absolute_percentage_error(y_test, y_pred_train) 
        max_coeff = ridge_regressor.coef_.round(3)
        eval_metrics_dictionary['r2'].append(infun_r2)
        eval_metrics_dictionary['MSE'].append(infun_MSE)
        eval_metrics_dictionary['RMSE'].append(infun_RMSE)
        eval_metrics_dictionary['MAPE'].append(infun_MAPE)
        """
        coeff_used = np.sum(model.coef_!=0)

        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        ridge_opt_dictionary['alpha'].append(alpha)
        ridge_opt_dictionary['intercept'].append(intercept)
        ridge_opt_dictionary['tol'].append(tol)
        ridge_opt_dictionary['solver'].append(solver)
        ridge_opt_dictionary['no_of_coefficients'].append(coeff_used)

        return mean_MSE*mean_MAPE*(1-mean_r2)*(1-mean_r2)
        #return mean_MSE

    study2 = optuna.create_study(study_name="RidgeRegression Optimization")
    study2.optimize(objective, n_trials=number_of_trials)
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    df_framework['combined_score'] = abs(abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']*df_framework['mean_MAPE'])
    df_bestvalue = df_framework[df_framework['combined_score']==min(df_framework['combined_score'])]
    return df_bestvalue

In [12]:
ridge_optimization(200)

[I 2023-09-01 13:30:04,558] A new study created in memory with name: RidgeRegression Optimization
[I 2023-09-01 13:30:04,589] Trial 0 finished with value: 73754.53185984762 and parameters: {'alpha': 31.691610658028857, 'fit_intercept': False, 'tol': 0.4532446649018952, 'solver': 'sag'}. Best is trial 0 with value: 73754.53185984762.
[I 2023-09-01 13:30:04,601] Trial 1 finished with value: 61368.876293216825 and parameters: {'alpha': 61.64269098255453, 'fit_intercept': True, 'tol': 0.01684876219009207, 'solver': 'cholesky'}. Best is trial 1 with value: 61368.876293216825.
[I 2023-09-01 13:30:04,615] Trial 2 finished with value: 60797.288798588816 and parameters: {'alpha': 6.324144071433002, 'fit_intercept': False, 'tol': 0.011421463748602896, 'solver': 'auto'}. Best is trial 2 with value: 60797.288798588816.
[I 2023-09-01 13:30:04,643] Trial 3 finished with value: 61520.59531617049 and parameters: {'alpha': 87.55893436252715, 'fit_intercept': True, 'tol': 0.0010841955747050106, 'solver'

,mean_r2,mean_MAPE,mean_MSE,std_r2,std_MAPE,std_MSE,score,combined_score
155,0.01427,2.605928,21309.386152,0.00753,0.250713,1767.610818,5,54738.318896


In [13]:
math.sqrt(12886.800806)

113.52004583332408

In [14]:
def RandomForestRegressor_optimization(number_of_trials):
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[]}
    randoforreg_dictionary = {
        'n_estimators': [], 
        'max_depth': [],
        'min_samples_split': [],
        'min_samples_leaf':[]
        }

    def objective(trial):
        ## HYPER PARAMETER TUNING
        n_estimators = trial.suggest_int("n_estimators", 10, 300, log=True)
        max_depth = trial.suggest_int("max_depth", 2, 32)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

        # Create and fit random forest model
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42,
            )
        model.fit(ridge_Xtrain, y_train)
        
        # Function parameters for cross_val_score; 
        #   - estimator - The model object to use to fit the data
        #   - X - The data to fit the model on
        #   - y - The target of the model
        #   - scoring - The error metric to use
        #   - cv - The number of splits to use
        
        # error score to use; 
        #   - accuracy
        #   - balanced_accuracy
        #   - roc_auc
        #   - f1
        #   - neg_mean_absolute_error
        #   - neg_root_mean_squared_error
        #   - r2
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        score = cross_validate(model, ridge_Xtrain, y_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_mean_squared_error'), cv=3)
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_mean_squared_error'])
        
        """
        y_pred_train= ridge_regressor.predict(X_test)
            
        Evaluation metrics;
        - R squared
        - MSE
        - RMSE
        - MAPE
        
        infun_MSE = mean_squared_error(y_test,y_pred_train)
        infun_RMSE = math.sqrt(infun_MSE)
        infun_r2 = r2_score(y_test, y_pred_train) 
        infun_MAPE = mean_absolute_percentage_error(y_test, y_pred_train) 
        max_coeff = ridge_regressor.coef_.round(3)
        eval_metrics_dictionary['r2'].append(infun_r2)
        eval_metrics_dictionary['MSE'].append(infun_MSE)
        eval_metrics_dictionary['RMSE'].append(infun_RMSE)
        eval_metrics_dictionary['MAPE'].append(infun_MAPE)
        """
        #coeff_used = np.sum(model.coef_!=0)

        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        randoforreg_dictionary['n_estimators'].append(n_estimators)
        randoforreg_dictionary['max_depth'].append(max_depth)
        randoforreg_dictionary['min_samples_split'].append(min_samples_split)
        randoforreg_dictionary['min_samples_leaf'].append(min_samples_leaf)
        #randoforreg_dictionary['no_of_coefficients'].append(coeff_used)

        return mean_MSE*mean_MAPE*(1-mean_r2)*(1-mean_r2)
        #return mean_MSE

    study2 = optuna.create_study(study_name="RandomForestRegression Optimization")
    study2.optimize(objective, n_trials=number_of_trials)
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    df_framework['combined_score'] = abs(abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']*df_framework['mean_MAPE'])
    df_bestvalue = df_framework[df_framework['combined_score']==min(df_framework['combined_score'])]
    return df_bestvalue

In [15]:
RandomForestRegressor_optimization(20)

[I 2023-09-01 13:30:13,542] A new study created in memory with name: RandomForestRegression Optimization
[I 2023-09-01 13:30:29,581] Trial 0 finished with value: 51741.532991626074 and parameters: {'n_estimators': 157, 'max_depth': 11, 'min_samples_split': 8, 'min_samples_leaf': 1}. Best is trial 0 with value: 51741.532991626074.
[I 2023-09-01 13:30:31,154] Trial 1 finished with value: 51187.44808971352 and parameters: {'n_estimators': 12, 'max_depth': 29, 'min_samples_split': 6, 'min_samples_leaf': 10}. Best is trial 1 with value: 51187.44808971352.
[I 2023-09-01 13:30:37,132] Trial 2 finished with value: 54782.58277697465 and parameters: {'n_estimators': 37, 'max_depth': 17, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 1 with value: 51187.44808971352.
[I 2023-09-01 13:30:41,279] Trial 3 finished with value: 49179.0413339604 and parameters: {'n_estimators': 35, 'max_depth': 13, 'min_samples_split': 4, 'min_samples_leaf': 7}. Best is trial 3 with value: 49179.041333960

,mean_r2,mean_MAPE,mean_MSE,std_r2,std_MAPE,std_MSE,score,combined_score
13,0.061617,2.677337,20290.476753,0.009114,0.111082,1723.892907,5,50977.126178


In [16]:
11989.482402

11989.482402

In [17]:
math.sqrt(11989.482402)

109.49649493020313

In [18]:

def SVRegressor_optimization(number_of_trials):
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[]}
    SVRegressor_dictionary ={
        'kernel' :[],
        'C' :[],
        'degree' :[],
        'coef0' :[],
        'gamma' :[]
    }
    def objective(trial):
        ## HYPER PARAMETER TUNING
        degree = trial.suggest_int("degree", 3, 8)
        C = trial.suggest_int("C", 1, 10)
        coef0 = trial.suggest_float("coef0", 0.01, 10)
        gamma = trial.suggest_categorical("gamma", ["auto", 'scale'])
        kernel = trial.suggest_categorical("kernel", ['poly'])
        # ['linear', 'poly', 'rbf', 'sigmoid'])


        # Create and fit random forest model
        model = SVR(
            kernel=kernel,
            C=C,
            degree=degree,
            coef0=coef0,
            gamma=gamma,
            )
        model.fit(ridge_Xtrain, y_train)
        
        # Function parameters for cross_val_score; 
        #   - estimator - The model object to use to fit the data
        #   - X - The data to fit the model on
        #   - y - The target of the model
        #   - scoring - The error metric to use
        #   - cv - The number of splits to use
        
        # error score to use; 
        #   - accuracy
        #   - balanced_accuracy
        #   - roc_auc
        #   - f1
        #   - neg_mean_absolute_error
        #   - neg_root_mean_squared_error
        #   - r2
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        score = cross_validate(model, ridge_Xtrain, y_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_mean_squared_error'), cv=3)
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_mean_squared_error'])
        
        """
        y_pred_train= ridge_regressor.predict(X_test)
            
        Evaluation metrics;
        - R squared
        - MSE
        - RMSE
        - MAPE
        
        infun_MSE = mean_squared_error(y_test,y_pred_train)
        infun_RMSE = math.sqrt(infun_MSE)
        infun_r2 = r2_score(y_test, y_pred_train) 
        infun_MAPE = mean_absolute_percentage_error(y_test, y_pred_train) 
        max_coeff = ridge_regressor.coef_.round(3)
        eval_metrics_dictionary['r2'].append(infun_r2)
        eval_metrics_dictionary['MSE'].append(infun_MSE)
        eval_metrics_dictionary['RMSE'].append(infun_RMSE)
        eval_metrics_dictionary['MAPE'].append(infun_MAPE)
        """
        #coeff_used = np.sum(model.coef_!=0)

        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        SVRegressor_dictionary['degree'].append(degree)
        SVRegressor_dictionary['C'].append(C)
        SVRegressor_dictionary['coef0'].append(coef0)
        SVRegressor_dictionary['gamma'].append(gamma)
        SVRegressor_dictionary['kernel'].append(kernel)
        #randoforreg_dictionary['no_of_coefficients'].append(coeff_used)
        #return mean_MSE*mean_MAPE*(1-mean_r2)*(1-mean_r2)
        return mean_MSE

    study2 = optuna.create_study(study_name="SVRRegression Optimization")
    study2.optimize(objective, n_trials=number_of_trials)
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    df_framework['combined_score'] = abs(abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']*df_framework['mean_MAPE'])
    df_bestvalue = df_framework[df_framework['combined_score']==min(df_framework['combined_score'])]
    return df_bestvalue

In [19]:
## when c == degreee it looks v good
SVRegressor_optimization(5)

[I 2023-09-01 13:36:13,395] A new study created in memory with name: SVRRegression Optimization


In [ ]:
def DecisionTreeRegressor_optimization(number_of_trials):
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[]}
    dectrereg_dictionary = {'max_depth': [], 'min_samples_split': [],'min_samples_leaf': [],'random_state':[]}

    def objective(trial):
        ## HYPER PARAMETER TUNING
        max_depth = trial.suggest_int("max_depth", 2, 32)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
        random_state = trial.suggest_int("random_state", 0, 50)

        # Create and fit random forest model
        model = DecisionTreeRegressor(
                max_depth=max_depth,
                min_samples_leaf=min_samples_leaf, 
                min_samples_split=min_samples_split,
                random_state=random_state
                )
        model.fit(ridge_Xtrain, y_train)
        
        # Function parameters for cross_val_score; 
        #   - estimator - The model object to use to fit the data
        #   - X - The data to fit the model on
        #   - y - The target of the model
        #   - scoring - The error metric to use
        #   - cv - The number of splits to use
        
        # error score to use; 
        #   - accuracy
        #   - balanced_accuracy
        #   - roc_auc
        #   - f1
        #   - neg_mean_absolute_error
        #   - neg_root_mean_squared_error
        #   - r2
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        score = cross_validate(model, ridge_Xtrain, y_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_mean_squared_error'), cv=3)
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_mean_squared_error'])
        
        """
        y_pred_train= ridge_regressor.predict(X_test)
            
        Evaluation metrics;
        - R squared
        - MSE
        - RMSE
        - MAPE
        
        infun_MSE = mean_squared_error(y_test,y_pred_train)
        infun_RMSE = math.sqrt(infun_MSE)
        infun_r2 = r2_score(y_test, y_pred_train) 
        infun_MAPE = mean_absolute_percentage_error(y_test, y_pred_train) 
        max_coeff = ridge_regressor.coef_.round(3)
        eval_metrics_dictionary['r2'].append(infun_r2)
        eval_metrics_dictionary['MSE'].append(infun_MSE)
        eval_metrics_dictionary['RMSE'].append(infun_RMSE)
        eval_metrics_dictionary['MAPE'].append(infun_MAPE)
        """
        #coeff_used = np.sum(model.coef_!=0)

        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        dectrereg_dictionary['max_depth'].append(max_depth)
        dectrereg_dictionary['min_samples_split'].append(min_samples_split)
        dectrereg_dictionary['min_samples_leaf'].append(min_samples_leaf)
        dectrereg_dictionary['random_state'].append(random_state)
        #randoforreg_dictionary['no_of_coefficients'].append(coeff_used)

        return mean_MSE*mean_MAPE*(1-mean_r2)*(1-mean_r2)
        #return mean_MSE

    study2 = optuna.create_study(study_name="DecisionTreeRegressor Optimization")
    study2.optimize(objective, n_trials=number_of_trials)
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    df_framework['combined_score'] = abs(abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']*df_framework['mean_MAPE'])
    df_bestvalue = df_framework[df_framework['combined_score']==min(df_framework['combined_score'])]
    return df_bestvalue

In [ ]:
DecisionTreeRegressor_optimization(200)